# Petition Results - Automated Download and Reporting

Created by Michael George (AKA Logiqx)

## Import Libraries

In [1]:
import urllib.request
import json
import os
import datetime

## Process History

In [2]:
def processHistory(attributes, f):
    '''Process history for a petition'''

    def getDateTime(dt):
        dtSplit = dt.split('T')
        dateStr = dtSplit[0]
        timeStr = dtSplit[1].split('.')[0]
        return '%s | %s' % (dateStr, timeStr)

    f.write('### History\n\n')
    f.write('| Event | Date | Time |\n')
    f.write('| - | - | - |\n')
    f.write('| Updated | %s |\n' % getDateTime(attributes['updated_at']))

    if (attributes['government_response_at']):
        f.write('| Government Response | %s |\n' % getDateTime(attributes['government_response_at']))
    if (attributes['debate_threshold_reached_at']):
        f.write('| Debate Threshold Reached | %s |\n' % getDateTime(attributes['debate_threshold_reached_at']))
    if (attributes['response_threshold_reached_at']):
        f.write('| Response Threshold Reached | %s |\n' % getDateTime(attributes['response_threshold_reached_at']))

    f.write('| Opened | %s |\n' % getDateTime(attributes['opened_at']))

    if (attributes['moderation_threshold_reached_at']):
        f.write('| Moderation Threshold Reached | %s |\n' % getDateTime(attributes['moderation_threshold_reached_at']))

    f.write('| Created | %s |\n' % getDateTime(attributes['created_at']))

## Process Countries

In [3]:
def processCountries(attributes, f):
    '''Process all of the countries for a petition'''

    # Count signature worldwide
    worldTotal = attributes['signature_count']

    # Count signatures for all of the countries
    total = 0
    for country in attributes['signatures_by_country']:
        total += country['signature_count']

    # Ensure that all signatures are accounted for by creating an 'Unknown' category
    diff = worldTotal - total
    if diff:
        if diff / worldTotal > 0.1:
            print('Missing country: %0.2f%%' % (diff / worldTotal))
        unknown = {'name': 'Unknown', 'code': 'ZZ', 'signature_count':  diff}
        attributes['signatures_by_country'].append(unknown)
        total += diff

    # List all of the countries sorted by signature count
    countries = sorted(attributes['signatures_by_country'], key=lambda country: country['signature_count'], reverse=True)

    f.write('### Signatures by Country\n\n')
    f.write('| # | Country | Signatures | % |\n')
    f.write('| - | - | -: | -: |\n')

    for i in range(len(countries)):
        country = countries[i]
        pct = 100 * country['signature_count'] / total
        f.write('| %d | %s | %s | %0.2f |\n' % (i + 1, country['name'], '{:,}'.format(country['signature_count']), pct))

## Process Constituencies

In [4]:
def processConstituencies(attributes, f):
    '''Process all of the constituencies for a petition'''

    # Count signatures for the UK (ISO code GB)
    ukTotal = 0
    for country in attributes['signatures_by_country']:
        if country['code'] == 'GB':
            ukTotal = country['signature_count']

    # Count signatures for all of the constituencies
    total = 0
    for constituency in attributes['signatures_by_constituency']:
        total += constituency['signature_count']

    # Ensure that all signatures are accounted for by creating an 'Unknown' category
    diff = ukTotal - total
    if diff:
        if diff / ukTotal > 0.1:
            print('Missing constituency: %0.2f%%' % (diff / ukTotal))
        unknown = {'name': 'Unknown', 'ons_code': 'Z99999999', 'mp': 'Unknown', 'signature_count':  diff}
        attributes['signatures_by_constituency'].append(unknown)
        total += diff

    # List all of the constituencies sorted by signature count
    constituencies = sorted(attributes['signatures_by_constituency'], key=lambda constituency: constituency['signature_count'], reverse=True)

    f.write('### Signatures by Constituency\n\n')
    f.write('| # | Constituency | MP | Signatures | % |\n')
    f.write('| - | - | - | -: | -: |\n')

    for i in range(len(constituencies)):
        constituency = constituencies[i]
        pct = 100 * constituency['signature_count'] / total
        f.write('| %d | %s | %s | %s | %0.2f |\n' % (i + 1, constituency['name'], constituency['mp'],
            '{:,}'.format(constituency['signature_count']), pct))

## Process Petition

In [5]:
def processPetition(state, petition):
    '''Process a single petition'''

    # Date / time of the refresh
    refreshTime = datetime.datetime.now()

    # Specify the URL
    url = petition['links']['self']

    # Open the URL and retrieve the JSON
    handle = urllib.request.urlopen(url)

    # Parse JSON
    petition = json.load(handle)

    # Everything of interest is under "attributes"
    petitionData = petition['data']
    petitionId = petitionData['id']
    attributes = petitionData['attributes']
    link = petition['links']['self'].replace('.json','')

    # Remove newlines from the action (petition title)
    attributes['action'] = attributes['action'].replace('\r\n',' ')

    # Write JSON to local file
    fn = "../data/%s/%d.json" % (state, petitionId)
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    with open(fn, 'w') as f:
        f.write(json.dumps(petition, indent=4))

    # Write summary to local file
    fn = "../docs/%s/%d.md" % (state, petitionId)
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    with open(fn, 'w') as f:
        f.write('# %s\n\n' % attributes['action'])
        f.write('### Background\n\n')
        f.write('%s\n\n' % attributes['background'])
        f.write('### Signatures\n\n')
        f.write('**%s signatures**\n\n' % '{:,}'.format(attributes['signature_count']))
        f.write('Data taken from [%s](%s)\n\n' % (link, link))
        f.write('Last refreshed %s\n\n' % refreshTime.strftime("%Y-%m-%d %H:%M:%S"))
        if (attributes['government_response']) is not None:
            f.write('### Government Response\n\n')
            f.write('%s\n\n' % attributes['government_response']['summary'])
            f.write('%s\n\n' % attributes['government_response']['details'])

        processHistory(attributes, f)
        f.write('\n')
        
        processCountries(attributes, f)
        f.write('\n')

        processConstituencies(attributes, f)

    return petition

## Process Petitions

In [6]:
def processPetitions(state):
    '''Process the top petitions'''

    # Date / time of the refresh
    refreshTime = datetime.datetime.now()

    # Specify the URL
    url = "https://petition.parliament.uk/petitions.json?state=" + state

    # Open the URL and retrieve the JSON
    handle = urllib.request.urlopen(url)

    # Parse JSON
    index = json.load(handle)

    # Write JSON to local file
    fn = "../data/%s/index.json" % state
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    with open(fn, 'w') as f:
        f.write(json.dumps(index, indent=4))

    # Write summary to local file
    fn = "../docs/%s/README.md" % state
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    with open(fn, 'w') as f:
        topX = len(index['data'])
        if (topX > 0):
            f.write('# Top %d %s Petitions\n\n' % (topX, state.capitalize()))
        else:
            f.write('# Top %s Petitions\n\n' % state.capitalize())

        f.write('Data taken from [UK Government and Parliament](%s)\n\n' % url.replace('.json', ''))
        f.write('Reports created using a simple [IPython Notebook](https://github.com/Logiqx/petition-stats)\n\n')
        f.write('Last refreshed %s\n\n' % refreshTime.strftime("%Y-%m-%d %H:%M:%S"))

        if (topX > 0):
            f.write('| Petition Name | Total Signatures | UK Signatures | Overseas Signatures | Unknown Signatures | UK Signatures % | Overseas Signatures % | Unknown Signatures % |\n')
            f.write('| - | -: | -: | -: | -: | -: | -: | -: |\n')

            # Process the top X petitions
            for i in range(topX):
                petition = index['data'][i]
                petition = processPetition(state, petition)
                attributes = petition['data']['attributes']

                total = attributes['signature_count']
                ukTotal = unknownTotal = 0
                for country in attributes['signatures_by_country']:
                    if country['code'] == 'GB':
                        ukTotal = country['signature_count']
                    if country['code'] == 'ZZ':
                        unknownTotal = country['signature_count']
                overseasTotal = total - ukTotal - unknownTotal

                f.write('| [%s](%s.md) | %s | %s | %s | %s | %0.2f | %0.2f | %0.2f |\n' % (
                    attributes['action'], petition['data']['id'], '{:,}'.format(total),
                    ('{:,}'.format(ukTotal)), ('{:,}'.format(overseasTotal)), ('{:,}'.format(unknownTotal)),
                    100 * ukTotal / total, 100 * overseasTotal / total, 100 * unknownTotal / total
                ))

    print('Processed %s petitions!' % state)

processPetitions("open")
processPetitions("closed")

Processed open petitions!
Processed closed petitions!
